**Step 1: Find all patients with condition X from three sources**

**Required**
* Connect to FHIR Synthetic Patient Data
* Connect to OHDSI Synthetic Patient Data
* Connect to HUSH+ Synthetic Patient Data

**Step 1.1: Find 

In [1]:
#connect to UMLS
import mysql.connector

try:
  cnx = mysql.connector.connect(user='tadmin',
                                password='ncats_translator!',
                                database='umls',
                                host='translator.ceyknq0yekb3.us-east-1.rds.amazonaws.com')
except mysql.connector.Error as err:
  if err.errno == errorcode.ER_ACCESS_DENIED_ERROR:
    print("Something is wrong with your user name or password")
  elif err.errno == errorcode.ER_BAD_DB_ERROR:
    print("Database does not exist")
  else:
    print(err)
else:
  print
  #cnx.close()

In [2]:
#Define functions
def findMeshAndICD10(name):
    cursor = cnx.cursor()
    query = ("SELECT CUI FROM umls.MRCONSO WHERE STR='"+name+"'")
    cursor.execute(query, ())
    res = "Undef"
    for code in cursor:
        if res=="Undef":
            res = code
    if res != "Undef":
        query = ("SELECT CODE FROM umls.MRCONSO WHERE SAB='MSH' AND CUI='"+res[0]+"'")
        cursor.execute(query, ())
        msh = "Undef"
        for code in cursor:
            msh = code
        query = ("SELECT CODE FROM umls.MRCONSO WHERE SAB='ICD10' AND CUI='"+res[0]+"'")
        cursor.execute(query, ())
        icd10 = "Undef"
        for code in cursor:
            icd10 = code
        query = ("SELECT CODE FROM umls.MRCONSO WHERE SAB='ICD10CM' AND CUI='"+res[0]+"'")
        cursor.execute(query, ())
        for code in cursor:
            icd10 = code
        return (msh[0],icd10[0])
    return ("Undef","Undef") 


In [3]:
findMeshAndICD10('Asthma')

(u'D001249', u'J45.90')

In [15]:
## FHIR patient data
import requests
base_url = 'http://ictrweb.johnshopkins.edu/rest/synthetic'
patients = base_url+"/Patient"+"?_count=999"

req = requests.get(patients)
print req.json()

{u'resourceType': u'Bundle', u'entry': [{u'extension': [{u'race': u'Asian'}], u'resourceType': u'Patient', u'gender': u'Female', u'birthDate': u'1986-08-30', u'address': [{u'postalCode': u'22209', u'city': u'Washington', u'use': u'home', u'line': u'', u'state': u'DC'}], u'active': True, u'identifier': [{u'value': u'491207340', u'label': u'patient id'}], u'fullurl': u'http://ictrweb.johnshopkins.edu/rest/synthetic/Patient/491207340'}, {u'extension': [{u'race': u'White'}], u'resourceType': u'Patient', u'gender': u'Female', u'birthDate': u'1975-05-12', u'address': [{u'postalCode': u'22207', u'city': u'Washington', u'use': u'home', u'line': u'', u'state': u'DC'}], u'active': True, u'identifier': [{u'value': u'517874786', u'label': u'patient id'}], u'fullurl': u'http://ictrweb.johnshopkins.edu/rest/synthetic/Patient/517874786'}, {u'extension': [{u'race': u'White'}], u'resourceType': u'Patient', u'gender': u'Female', u'birthDate': u'1973-05-08', u'address': [{u'postalCode': u'20015', u'city'